In [3]:
#******************************************************************************************************#
#Course: ISDS 558 - Advanced Software Development with Web Applications
#Term: Spring 2025
#Instructor: Daoji Li, Ph.D.
#Group: 3
#Members: Keerthanaa Ellur , Duy Do , Jason Ho , Tin Nguyen, Zoe Shahbaz
#
#Project Title: Flask User Registration & Login System with SMS Verification
#Description:
#   This project is a secure web-based user authentication system built using Flask and Flask-WTF.
#   It allows users to register and log in with real-time form validation and password strength feedback.
#   The system includes the following features:

#   - User Registration with username, email, phone, and password input
#   - Password validation with live rule checks (length, special character, number, etc.)
#   - Email and username uniqueness check
#   - Terms of Service agreement checkbox with linked ToS page
#   - SMS code-based verification step (simulated via console print)
#   - Login system with two-step verification using a new code per login
#   - Session handling using Flask's secure cookies
#   - Data storage using JSON files for pending and registered users
#   - Public URL exposure using ngrok for testing in cloud environments like Google Colab
#
#   This system is suitable for educational or prototype use and can be extended with real SMS APIs
#   like Twilio, database integration, and additional security features for production environments.
#******************************************************************************************************#


#------------------------------------------------------------------------------------------------#
#List of steps that run in this project:
#------------------------------------------------------------------------------------------------#
# Step 1: Install required packages
# Step 2: Import external modules and Python standard libraries
# Step 3: Download existing registered users file from Dropbox and save locally
# Step 4: Configure ngrok with auth token
# Step 5: Configure Flask app and secret key
# Step 6: Define custom password validator function
# Step 7: Define Flask-WTF forms for Register, Login, and Verify Code
# Step 8: Create HTML templates for Register, Login, Verify, Welcome, Terms
# Step 9: Define route for registration form and handle form validation + SMS code generation
# Step 10: Define route to verify registration code from SMS
# Step 11: Define route to handle login and trigger SMS code for verification
# Step 12: Define route to verify login code from SMS
# Step 13: Define welcome page route after successful login or registration
# Step 14: Define Terms of Service page
# Step 15: Define mock function to simulate sending SMS verification code
# Step 16: Start Flask app using ngrok for public access
#------------------------------------------------------------------------------------------------#


#-----------------------------   MAIN CODE BEGIN -----------------------------#

#----------------------------------------------------------------------------------------------------------------------------#
# Install required packages (Flask, Flask-WTF for form handling, pyngrok for tunneling, email-validator for email validation)
#----------------------------------------------------------------------------------------------------------------------------#
!pip install flask flask-wtf pyngrok email-validator --quiet

#---------------------------------------------#
# Standard library and external module imports
#---------------------------------------------#
from flask import Flask, render_template_string, request, redirect, url_for, session  # Flask core and session management
from flask_wtf import FlaskForm  # Flask-WTF extension for handling forms
from wtforms import StringField, PasswordField, BooleanField, SubmitField  # Input fields
from wtforms.validators import DataRequired, Email, EqualTo, Length, ValidationError  # Form validators
from pyngrok import ngrok, conf  # pyngrok to create a public URL
import json, os, random, re  # Standard libraries for file handling, randomness, and regex

#------------------------------------------------------------#
# Download the latest registered_users.json file from Dropbox
#------------------------------------------------------------#
import requests
dropbox_url = "https://www.dropbox.com/scl/fi/cy8r3279msbjq7njwu39w/registered_users.json?rlkey=r8wds3pqre8osrftdr9qkgj0p&st=hvbk6g3s&dl=1"
save_path = "/content/registered_users.json"

# Check if the file downloads successfully, otherwise print an error
response = requests.get(dropbox_url)
if response.status_code == 200 and response.content:
    with open(save_path, 'wb') as f:
        f.write(response.content)
    print("✅ File downloaded to", save_path)
else:
    print("❌ Failed to download file. Check the link or permissions.")

#-------------------------------#
# Set ngrok authentication token
#-------------------------------#
conf.get_default().auth_token = "2vQ3V42htlY7gXjMpIeLN8u4HX9_79gAVSiJauDkdUNdwxFwy"

# Flask app configuration
app = Flask(__name__)
app.config['SECRET_KEY'] = 'supersecretkey'  # Used for securely signing session cookies
app.config['WTF_CSRF_ENABLED'] = False  # Disable CSRF for development/testing (not recommended for production)

#------------------------------------#
# Custom password strength validator
#------------------------------------#
def strong_password(form, field):
    password = field.data
    username = form.username.data.lower() if form.username.data else ''
    email = form.email.data.lower() if form.email.data else ''
    errors = []
    if len(password) < 6: #password at least 6 characters
        errors.append("at least 6 characters")
    if not re.search(r'\d', password): #password contains at least 1 number
        errors.append("a number")
    if not re.search(r'[!@#$%^&*(),.?\":{}|<>]', password): #password contains at least 1 special characters
        errors.append("a special character")
    if username and username in password.lower(): #password cannot be similar to username
        errors.append("not be similar to your username")
    if email and email.split('@')[0] in password.lower(): #password cannot be similar to email
        errors.append("not be similar to your email")
    if errors:
        raise ValidationError("Password must contain " + ", ".join(errors))

#----------------------------------------------------------------------#
# Define Flask-WTF Forms for registration, login, and code verification
#----------------------------------------------------------------------#

# Register Form:
class RegistrationForm(FlaskForm):
    username = StringField('Username', [Length(min=4, max=20)])
    email = StringField('Email Address', [Email(), Length(min=10, max=200)])
    phone = StringField('Mobile Number', [DataRequired(), Length(min=10, max=15)])
    password = PasswordField('Password', [
        DataRequired(),
        strong_password,
        EqualTo('confirm', message="Passwords must match")
    ])
    confirm = PasswordField('Confirm Password')
    accept_tos = BooleanField('I accept the Terms of Service', [DataRequired()])
    submit = SubmitField('Register')

# Login Form
class LoginForm(FlaskForm):
    identifier = StringField('Username or Email', [DataRequired()])
    password = PasswordField('Password', [DataRequired()])
    submit = SubmitField('Login')

# Verify Form
class VerifyForm(FlaskForm):
    code = StringField('Enter Code', [DataRequired(), Length(min=6, max=6)])
    submit = SubmitField('Verify')

#---------------------------------------------------------------------------------------------------------------------------#
# Templates (register_template, login_template, verify_template, welcome_template) are HTML strings rendered with form data
#---------------------------------------------------------------------------------------------------------------------------#

#Register HTML template
register_template = """
<!DOCTYPE html><html><head><title>Register</title>
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet"></head>
<body class="bg-light"><div class="container mt-5">
<div class="card p-4 shadow mx-auto" style="max-width:500px">
<h2 class="mb-4 text-center">Register</h2>
{% if error %}<div class="alert alert-danger">{{ error }}</div>{% endif %}
<form method="POST">{{ form.hidden_tag() }}
{% for field in [form.username, form.email, form.phone] %}
<div class="mb-3">{{ field.label }} {{ field(class="form-control") }}
{% for error in field.errors %}<div class="text-danger">{{ error }}</div>{% endfor %}</div>{% endfor %}
<div class="mb-3">
  {{ form.password.label }}
  {{ form.password(class="form-control", id="password") }}
  <div class="form-check mt-2">
    <input class="form-check-input" type="checkbox" id="togglePassword">
    <label class="form-check-label" for="togglePassword">Show Password</label>
  </div>
  <ul class="list-unstyled mt-2" id="password-rules">
    <li id="rule-length" class="text-danger">• At least 6 characters</li>
    <li id="rule-number" class="text-danger">• Contains a number</li>
    <li id="rule-special" class="text-danger">• Contains a special character</li>
    <li id="rule-username" class="text-danger">• Not similar to username</li>
    <li id="rule-email" class="text-danger">• Not similar to email</li>
  </ul>
  {% for error in form.password.errors %}<div class="text-danger">{{ error }}</div>{% endfor %}
</div>
<div class="mb-3">{{ form.confirm.label }} {{ form.confirm(class="form-control") }}
{% for error in form.confirm.errors %}<div class="text-danger">{{ error }}</div>{% endfor %}</div>
{{ form.accept_tos(class="form-check-input") }}
<label class="form-check-label">
  I accept the <a href="/terms" target="_blank">Terms of Service</a>
</label>

{{ form.submit(class="btn btn-primary w-100") }}</form>
<div class="text-center mt-3">Already registered? <a href="/login">Login</a></div></div></div>

<script>
document.addEventListener('DOMContentLoaded', () => {
  const passwordInput = document.getElementById('password');
  const usernameInput = document.getElementsByName('username')[0];
  const emailInput = document.getElementsByName('email')[0];

  passwordInput.addEventListener('input', () => {
    const password = passwordInput.value;
    const username = usernameInput?.value?.toLowerCase() || '';
    const emailName = (emailInput?.value?.split('@')[0] || '').toLowerCase();

    document.getElementById('rule-length').className = password.length >= 6 ? 'text-primary' : 'text-danger';
    document.getElementById('rule-number').className = /\\d/.test(password) ? 'text-primary' : 'text-danger';
    document.getElementById('rule-special').className = /[!@#$%^&*(),.?":{}|<>]/.test(password) ? 'text-primary' : 'text-danger';
    document.getElementById('rule-username').className = username && password.toLowerCase().includes(username) ? 'text-danger' : 'text-primary';
    document.getElementById('rule-email').className = emailName && password.toLowerCase().includes(emailName) ? 'text-danger' : 'text-primary';
  });

  document.getElementById('togglePassword').addEventListener('change', function () {
    passwordInput.type = this.checked ? 'text' : 'password';
  });
});
</script>

</body></html>
"""

#Login HTML template
login_template = """
<!DOCTYPE html><html><head><title>Login</title>
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet"></head>
<body class="bg-light"><div class="container mt-5">
<div class="card p-4 shadow mx-auto" style="max-width:400px">
<h2 class="mb-4 text-center">Login</h2>
{% if error %}<div class="alert alert-danger">{{ error }}</div>{% endif %}
<form method="POST">{{ form.hidden_tag() }}
<div class="mb-3">{{ form.identifier.label }} {{ form.identifier(class="form-control") }}</div>
<div class="mb-3">
  {{ form.password.label }}
  {{ form.password(class="form-control", id="login-password") }}
  <div class="form-check mt-2">
    <input class="form-check-input" type="checkbox" id="toggleLoginPassword">
    <label class="form-check-label" for="toggleLoginPassword">Show Password</label>
  </div>
</div>
{{ form.submit(class="btn btn-primary w-100") }}</form>
<div class="text-center mt-3">Don't have an account? <a href="/">Register</a></div></div></div>

<script>
document.addEventListener('DOMContentLoaded', () => {
  const loginPassword = document.getElementById('login-password');
  const toggleLogin = document.getElementById('toggleLoginPassword');
  if (loginPassword && toggleLogin) {
    toggleLogin.addEventListener('change', function () {
      loginPassword.type = this.checked ? 'text' : 'password';
    });
  }
});
</script>
</body></html>
"""

#Verify HTML template
verify_template = """
<!DOCTYPE html><html><head><title>Verify</title>
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet"></head>
<body class="bg-warning bg-opacity-10"><div class="container mt-5">
<div class="card p-4 shadow mx-auto" style="max-width:400px">
<h2 class="mb-4 text-center">Verify Phone</h2>
{% if sms_code %}
<div class="alert alert-info text-center">
    🛑 For testing: your SMS code is <strong>{{ sms_code }}</strong>
</div>
{% endif %}
{% if error %}<div class="alert alert-danger">{{ error }}</div>{% endif %}
<form method="POST">{{ form.hidden_tag() }}
<div class="mb-3">{{ form.code.label }} {{ form.code(class="form-control") }}
{% for error in form.code.errors %}<div class="text-danger">{{ error }}</div>{% endfor %}</div>
{{ form.submit(class="btn btn-success w-100") }}</form></div></div></body></html>
"""
#Welcome HTML template
welcome_template = """
<!DOCTYPE html><html><head><title>Welcome</title>
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet"></head>
<body class="bg-success bg-opacity-10"><div class="container mt-5 text-center">
<div class="alert alert-success shadow-lg rounded-4 p-4">
<h2>🎉 Welcome, {{ username }}!</h2>
<p>You are now logged in.</p>
<a href="/login" class="btn btn-outline-primary">Logout</a></div></div></body></html>
"""

#-------------------------------------#
# Route: Register
#-------------------------------------#
@app.route('/', methods=['GET', 'POST'])
def register():
    form = RegistrationForm()
    error = None
    if form.validate_on_submit():
        try:
            # Check for existing user by email or username
            if os.path.exists('/content/registered_users.json'):
                with open('/content/registered_users.json', 'r') as f:
                    for line in f:
                        existing_user = json.loads(line)
                        if form.username.data.lower() == existing_user['username'].lower():
                            error = "Username has already been registered."
                            return render_template_string(register_template, form=form, error=error)
                        if form.email.data.lower() == existing_user['email'].lower():
                            error = "Email has already been registered."
                            return render_template_string(register_template, form=form, error=error)

            # Proceed with registration, generate verification code, and save temporarily
            code = str(random.randint(100000, 999999))
            user_data = {
                'username': form.username.data,
                'email': form.email.data,
                'phone': form.phone.data,
                'password': form.password.data,
                'code': code
            }
            with open('/content/pending_user.json', 'w') as f:
                json.dump(user_data, f)
            send_sms_code(form.phone.data, code) # Simulate SMS
            session['sms_code'] = code
            return redirect(url_for('verify'))
        except Exception as e:
            error = f"Registration error: {str(e)}"
    return render_template_string(register_template, form=form, error=error)

#---------------------------------------------#
# Route: Phone verification after registration
#---------------------------------------------#
@app.route('/verify', methods=['GET', 'POST'])
def verify():
    form = VerifyForm()
    error = None
    if form.validate_on_submit():
        with open('/content/pending_user.json', 'r') as f:
            user_data = json.load(f)
        if form.code.data == user_data['code']:
            with open('/content/registered_users.json', 'a') as f:
                f.write(json.dumps(user_data) + '\n')
            os.remove('/content/pending_user.json') # Remove temp data
            session.pop('sms_code', None)
            return redirect(url_for('welcome', username=user_data['username']))
        else:
            error = "Invalid verification code"
    return render_template_string(verify_template, form=form, error=error, sms_code=session.get('sms_code'))

#---------------------#
# Route: Login form
#---------------------#
@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    error = None
    if form.validate_on_submit():
        identifier = form.identifier.data.lower()
        password = form.password.data
        found = False

        if os.path.exists('/content/registered_users.json'):
            with open('/content/registered_users.json', 'r') as f:
                for line in f:
                    user = json.loads(line)
                    if user['email'].lower() == identifier or user['username'].lower() == identifier:
                        found = True
                        if user['password'] == password:
                            code = str(random.randint(100000, 999999))
                            user['code'] = code
                            with open('/content/pending_login.json', 'w') as pf:
                                json.dump(user, pf)
                            send_sms_code(user['phone'], code)   # Simulate SMS
                            session['sms_code'] = code
                            return redirect(url_for('verify_login'))
                        else:
                            error = "Incorrect password"
                        break

        if not found:
            error = "Your account is not registered yet. Please register your user."

    return render_template_string(login_template, form=form, error=error)

#-------------------------------#
# Route: Verify code after login
#-------------------------------#
@app.route('/verify-login', methods=['GET', 'POST'])
def verify_login():
    form = VerifyForm()
    error = None
    if form.validate_on_submit():
        with open('/content/pending_login.json', 'r') as f:
            user_data = json.load(f)
        if form.code.data == user_data['code']:
            os.remove('/content/pending_login.json')
            session.pop('sms_code', None)
            return redirect(url_for('welcome', username=user_data['username']))
        else:
            error = "Invalid login verification code"
    return render_template_string(verify_template, form=form, error=error, sms_code=session.get('sms_code'))

#------------------------------------------------------------#
# Route: Welcome page after successful registration or login
#------------------------------------------------------------#
@app.route('/welcome')
def welcome():
    username = request.args.get('username', 'User')
    return render_template_string(welcome_template, username=username)

#-----------------------------#
# Route: Terms of Service page
#-----------------------------#
@app.route('/terms')
def terms():
    return render_template_string("""
    <!DOCTYPE html><html><head><title>Terms of Service</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet"></head>
    <body class="bg-light"><div class="container mt-5">
    <div class="card p-4 shadow mx-auto" style="max-width:800px">
    <h2>Terms of Service</h2>
    <p>By using this service, you agree to the following terms:</p>
    <ul>
        <li>You are responsible for the accuracy of the information you provide.</li>
        <li>All activities must comply with applicable laws and regulations.</li>
        <li>We reserve the right to modify or terminate the service at any time.</li>
        <li>Personal data may be used for verification and internal use only.</li>
    </ul>
    <p>If you do not agree to these terms, do not use the service.</p>
    <a href="/" class="btn btn-outline-primary">Back to Register</a>
    </div></div></body></html>
    """)

#------------------------------------------------------------------------------#
# Mock function for sending SMS (prints to console instead of sending real SMS)
#------------------------------------------------------------------------------#
def send_sms_code(phone, code):
    print(f"📱 [MOCK SMS] Your code is {code} (would be sent to {phone})")

#-----------------------------------------------------#
# Start the Flask development server with ngrok tunnel
#-----------------------------------------------------#
app.debug = True
public_url = ngrok.connect(5000)
print("🔗 Public URL:", public_url)
app.run(port=5000, use_reloader=False)

#-----------------------------   MAIN CODE END -----------------------------#

✅ File downloaded to /content/registered_users.json
🔗 Public URL: NgrokTunnel: "https://ea93-34-139-201-129.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 16:37:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 16:37:57] "GET /favicon.ico HTTP/1.1" 404 -
